In [6]:
import pandas as pd
from docx.api import Document

from tqdm import tqdm_notebook

In [4]:
document = Document('plan_obs/bsa_dkr_2019_3.docx')
table = document.tables[0]

In [7]:
data = []

keys = None
for i, row in tqdm_notebook(enumerate(table.rows)):
    text = (cell.text for cell in row.cells)

    if i == 0:
        keys = tuple(text)
        continue
    row_data = dict(zip(keys, text))
    data.append(row_data)

df = pd.DataFrame(data)
df.to_csv('test.csv', sep=" ", header=True, index=False)

In [8]:
# How get element of df:
# iloc[row,columns]
# df.iloc[0,:].tolist() # [5,6,7,8]  - row index 0
# df.iloc[:,0].tolist() # [5,9,13,17]  - column index 0
# df.iloc[0,0] # 5  - cell(0,0)
# df.iloc[1:,2].tolist() # [11,15,19]  - column index 2, but skip first row# iloc[row,columns]
print(df.iloc[0,0])

In [9]:
df

,"Казанцев, Потапов БСА, ЦАС, авто","Кийков, Малов И.Ф., Малов О.И. БСА,ЦАС-16, оператор","Лосовский, Думский, Авраменко БСА, ЦАС, авто","Малофеев, Теплых, Малов О. БСА, ЦАС, авто","Пширков, Потапов БСА, ЦАС, авто","Самодуров, БСА,ЦАС, авто","Сулейманова БСА, ДКР, ЦАС авто+оператор","Тюльбашев БСА,ЦАС-16, оператор"
0,,,,,,,,Круглосуточный обзор \n00h00m -24h00m\n11-15.0...
1,J0007+7303 (B0007+73)\nкопии от Малофеева\n,,,"J0007+7303 (B0007+73)\n01-10,16-25.07,\n01-10,...",,,,
2,"J0014+4746 (B0011+47)\n01-10,16-25,07,\n01-10,...",,,,,,,
3,"B0031-07,\n01-10,16-25,07,\n01-10,16-26.08,\n0...",,,,,,,
4,"B0052+51\n16-25,07,\n16-26.08,\n16-25.09",,,,,,,
5,,,,"0054+66\n01-10.07,\n01-10.08,\n01-10.09,",,,,
6,"B0105+65,\n16-25,07,\n16-26.08,\n16-25.09",,,,,,,
7,,,,"0122+14\n01-10,16-25,07,\n01-10,16-26.08,\n01-...",,,,
8,,,,,,,,
9,,,,,"B0138+59,\n07-08,20-22.06\n05-06,23-25.07\n08-...",,,


Случай, которые требуется рассмотреть:

+ Даты указаны диапазоном
+ Даты указаны перечислением
+ Только копии
+ Копии + диапазон + перечисление
+ Ошибки в указании дат (исключение, обход)
+ Сохранение в формате имя пульсара и кортеж типа 'DD.MM.YYYY'

In [16]:
df.columns

Index(['Казанцев, Потапов\nБСА, ЦАС, авто\n',
       'Кийков, Малов И.Ф., Малов О.И.\nБСА,ЦАС-16, оператор',
       'Лосовский, Думский, Авраменко\nБСА, ЦАС, авто',
       'Малофеев, \nТеплых, Малов О.\nБСА, ЦАС, авто',
       'Пширков, Потапов\nБСА, ЦАС, авто', 'Самодуров,\nБСА,ЦАС,\nавто',
       'Сулейманова\nБСА, ДКР, ЦАС\nавто+оператор',
       'Тюльбашев\nБСА,ЦАС-16, оператор'],
      dtype='object')

In [19]:
df.iloc[2][0]

'J0014+4746 (B0011+47)\n01-10,16-25,07,\n01-10,16-26.08,\n01-10,16-25.09 '

In [21]:
# Разделение по /n
cell = df.iloc[2][0].split('\n')
print(cell)

['J0014+4746 (B0011+47)', '01-10,16-25,07,', '01-10,16-26.08,', '01-10,16-25.09 ']


In [60]:
# последний элемент в таком формате всегда месяц
name_pulsar = cell[0]
for idx, value in enumerate(cell[1:]):
    value = value.replace('.', ',')  # заменяем точки на запятые для разделения данных
    value = value.split(',')  # разделяем данные
    value = [i.replace(' ', '') for i in value if i] # убираем из выдачи пустые строки
    print(value)

['01-10', '16-25', '07']
['01-10', '16-26', '08']
['01-10', '16-25', '09']


In [61]:
def get_days(arr):
    res_arr = []
    for idx, value in enumerate(arr[:-1]):
        if '-' not in value:
            res_arr.append(value + '.' + arr[-1])
        else:
            first, second = value.split('-')
            res_arr.extend(["%.2d" % i + '.' + arr[-1] for i in range(int(first), int(second) + 1)])
    return res_arr

In [62]:
get_days(value)

['01.09',
 '02.09',
 '03.09',
 '04.09',
 '05.09',
 '06.09',
 '07.09',
 '08.09',
 '09.09',
 '10.09',
 '16.09',
 '17.09',
 '18.09',
 '19.09',
 '20.09',
 '21.09',
 '22.09',
 '23.09',
 '24.09',
 '25.09']

In [56]:
int('01')

1